In [81]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, 'code')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np

#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = loadExcel('input/input.xlsx', N=9)

In [129]:
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('parameters.yaml') as f:
    params = yaml.safe_load(f)
    
try:
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetNumberOfCars(df,writer,params)
    writer.close()
except Exception as e:
    print(e)
    writer.close()

In [75]:
xd = df["License Plate Number"].value_counts()
xd['unknown'] = 0 
xd = xd[xd > 1]



df_multiple = df[df['License Plate Number'].isin(xd.index)]
df_single = df[~df['License Plate Number'].isin(xd.index)] 

In [76]:
minn = df_multiple.sort_values('Capture Time').groupby('License Plate Number').first().reset_index()
maxx = df_multiple.sort_values('Capture Time').groupby('License Plate Number').last().reset_index()

minnmaxx = minn.join(maxx, lsuffix='_minn', rsuffix='_maxx')

pd.crosstab(minnmaxx['Place combined index_minn'], minnmaxx['Place combined index_maxx'])


Place combined index_maxx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Place combined index_minn,,,,,,,,,,,,,,,,,,
1,11,126,185,63,1,12,1,27,5,29,16,113,5,30,3,13,3,45
2,61,6,18,8,4,1,0,1,0,2,0,4,1,3,1,1,0,1
3,14,290,113,880,13,88,11,122,43,198,26,248,26,73,25,97,27,309
4,29,214,333,57,77,6,12,14,25,24,2,4,4,6,8,3,24,11
5,7,84,13,156,69,587,20,179,37,110,13,49,11,31,11,82,7,27
6,11,15,141,27,784,65,87,17,78,13,4,7,1,4,10,7,15,4
7,6,48,16,114,8,170,32,680,17,159,14,69,9,28,18,140,1,16
8,4,31,28,34,79,23,301,54,41,18,1,3,2,2,8,16,3,3
9,7,76,28,293,15,95,21,261,43,815,61,743,31,91,46,194,8,25


,License Plate Number,Capture Time,Place sheet name
Place combined index,,,
1,1107,1107,1107
2,1048,1048,1048
3,2060,2060,2060
4,1567,1567,1567
5,468,468,468
6,674,674,674
7,458,458,458
8,455,455,455
9,975,975,975


In [108]:
# df.set_index('Capture Time').between_time(startTime, endTime).reset_index()

# separate dataset into two parts based on license plate count
plate_counts = df["License Plate Number"].value_counts()
plate_counts['unknown'] = 0  # fake unknown values, so they are selected into single dataset
plate_count = plate_counts[plate_counts > 1]

df_multiple = df[df['License Plate Number'].isin(plate_count.index)]
df_single = df[df['License Plate Number'].isin(plate_count.index)]

# create cross tab from multiple data
df_multiple_first = df_multiple.sort_values('Capture Time').groupby('License Plate Number').first().reset_index()
df_multiple_last = df_multiple.sort_values('Capture Time').groupby('License Plate Number').last().reset_index()

df_multiple_firstlast = df_multiple_first.join(df_multiple_last, lsuffix='_first', rsuffix='_last')

crosstab = pd.crosstab(df_multiple_firstlast['Place combined index_first'],
                       df_multiple_firstlast['Place combined index_last'])

# create half cross tab from single data
halfcrosstab = df_single.groupby('Place combined index').count()
halfcrosstab = halfcrosstab.drop(columns=['Capture Time', 'Place sheet name'])
halfcrosstab_to = halfcrosstab[halfcrosstab.index % 2 == 1].T
halfcrosstab_from = halfcrosstab[halfcrosstab.index % 2 == 0].T

In [109]:
halfcrosstab_to

Place combined index,1,3,5,7,9,11,13,15,17
License Plate Number,1561,6089,3843,3005,7840,3291,4284,2985,2039


In [105]:
halfcrosstab_to

,License Plate Number
Place combined index,
1,1561
3,6089
5,3843
7,3005
9,7840
11,3291
13,4284
15,2985
17,2039
